In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# features/ csv -> dataframe
feat_edu_raw = pd.read_csv("./birth_csv/feat_edu.csv")
df_edu = pd.DataFrame(feat_edu_raw)
feat_hou_raw = pd.read_csv("./birth_csv/feat_hou.csv")
df_hou = pd.DataFrame(feat_hou_raw)
feat_mar_raw = pd.read_csv("./birth_csv/feat_mar.csv")
df_mar = pd.DataFrame(feat_mar_raw)
feat_preg_raw = pd.read_csv("./birth_csv/feat_preg.csv")
df_preg = pd.DataFrame(feat_preg_raw)
feat_sal_raw = pd.read_csv("./birth_csv/feat_sal.csv")
df_sal = pd.DataFrame(feat_sal_raw)

# target/ csv -> dataframe
target_birth_raw = pd.read_csv("./birth_csv/target_birth.csv")
df_birth = pd.DataFrame(target_birth_raw)

# 'ADD_UP'이 포함된 데이터 제거
df_birth = df_birth[~df_birth['AREA'].str.contains('ADD_UP', na=False)]

# 고유한 지역 리스트 추출
areas = df_birth['AREA'].unique()

# feature dict / target dict
features_dict = {}
target_dict = {}

In [2]:
# 각 지역별 feature와 target 데이터 생성
for area in areas:
    area_target = df_birth[df_birth['AREA'] == area].drop(columns='AREA').transpose()
    area_target.columns = ['TARGET']

    # 각 feature 데이터 생성
    area_feat_edu = df_edu[df_edu['AREA'] == area].drop(columns='AREA').transpose()
    area_feat_edu.columns = ['EDU']
    
    area_feat_hou = df_hou[df_hou['AREA'] == area].drop(columns='AREA').transpose()
    area_feat_hou.columns = ['HOU']
    
    area_feat_mar = df_mar[df_mar['AREA'] == area].drop(columns='AREA').transpose()
    area_feat_mar.columns = ['MAR']
    
    area_feat_preg = df_preg[df_preg['AREA'] == area].drop(columns='AREA').transpose()
    area_feat_preg.columns = ['PREG']
    
    area_feat_sal = df_sal[df_sal['AREA'] == area].drop(columns='AREA').transpose()
    area_feat_sal.columns = ['SAL']

    # 지역별 feature 데이터 결합
    area_features = pd.concat([area_feat_edu, area_feat_hou, area_feat_mar, area_feat_preg, area_feat_sal], axis=1)

    # 'ADD_UP'이 포함된 features 데이터 제거
    area_features = area_features[~area_features.index.str.contains('ADD_UP', na=False)]

    # dictionary에 feature와 target 저장
    features_dict[area] = area_features
    target_dict[area] = area_target

In [3]:
# 각 지역에 대해 학습 및 평가 진행
for area in features_dict.keys():
    print(f"\n{area} 데이터에 대한 학습 시작")

    # 해당 지역의 feature와 target을 가져오기
    X_area = features_dict[area]
    y_area = target_dict[area]

    # 데이터 스케일링
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_area)

    # train_test_split 80% : 20%
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_area, test_size=0.2, random_state=42)

    # Lasso 회귀 모델 (다항 특성 및 GridSearch 없이)
    lasso_model = Lasso(alpha=0.1)  # alpha 값은 예시로 0.1 설정
    lasso_model.fit(X_train, y_train)

    # 예측
    y_train_pred = lasso_model.predict(X_train)
    y_test_pred = lasso_model.predict(X_test)

    # MSE (Mean Squared Error) for Test Data
    mse = mean_squared_error(y_test, y_test_pred)
    print(f"{area} - 평균 제곱 오차 (MSE) for Test Data: {mse}")

    # R² for Test Data
    r2_test = r2_score(y_test, y_test_pred)
    print(f"{area} - 결정 계수 (R²) for Test Data: {r2_test}")

    # MAE (Mean Absolute Error) for Test Data
    mae = mean_absolute_error(y_test, y_test_pred)
    print(f"{area} - 평균 절대 오차 (MAE) for Test Data: {mae}")

    # R² for Train Data
    r2_train = r2_score(y_train, y_train_pred)
    print(f"{area} - 결정 계수 (R²) for Train Data: {r2_train}")
    print("-" * 50)  # 구분선


SEOUL 데이터에 대한 학습 시작
SEOUL - 평균 제곱 오차 (MSE) for Test Data: 34052719.8373728
SEOUL - 결정 계수 (R²) for Test Data: 0.9386937525850316
SEOUL - 평균 절대 오차 (MAE) for Test Data: 4854.685083815748
SEOUL - 결정 계수 (R²) for Train Data: 0.9782571237698581
--------------------------------------------------

BUSAN 데이터에 대한 학습 시작
BUSAN - 평균 제곱 오차 (MSE) for Test Data: 1091627.913235973
BUSAN - 결정 계수 (R²) for Test Data: 0.9698836151884017
BUSAN - 평균 절대 오차 (MAE) for Test Data: 830.2604928503265
BUSAN - 결정 계수 (R²) for Train Data: 0.9639699254347736
--------------------------------------------------

DAEGU 데이터에 대한 학습 시작
DAEGU - 평균 제곱 오차 (MSE) for Test Data: 2584233.1286746287
DAEGU - 결정 계수 (R²) for Test Data: 0.8938964029288436
DAEGU - 평균 절대 오차 (MAE) for Test Data: 1362.148081139428
DAEGU - 결정 계수 (R²) for Train Data: 0.9819128887438341
--------------------------------------------------

INCHEON 데이터에 대한 학습 시작
INCHEON - 평균 제곱 오차 (MSE) for Test Data: 1022387.7219023902
INCHEON - 결정 계수 (R²) for Test Data: 0.9640464

c:\Users\user\miniconda3\envs\project_ml\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.658e+06, tolerance: 4.445e+05
  model = cd_fast.enet_coordinate_descent(
c:\Users\user\miniconda3\envs\project_ml\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.113e+06, tolerance: 3.557e+04
  model = cd_fast.enet_coordinate_descent(
c:\Users\user\miniconda3\envs\project_ml\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 